In [1]:
import tensorflow as tf
from tensorflow import keras
import sqlite3
import pandas as pd
from sklearn.model_selection import train_test_split
from datetime import datetime
import os
from logger import Logger

In [4]:
logger = Logger('../9. Model Building/model_building.log')
today = datetime.today().strftime("%Y\\%m\\%d")

In [5]:
logger.log(f'Building Neural network model for churn dataset on {today}')

In [6]:
# Connect to SQLite and load table
conn = sqlite3.connect("../feature_store.db")
try:
    df = pd.read_sql("SELECT * FROM feature_store;", conn)
    logger.log('fetched data from feature_store.db')
except:
    logger.log('failed to read data from feature_store')
conn.close()

In [7]:
# Drop ID column (not useful for training)
X = df.drop(columns=["customerID", "Churn"])
y = df["Churn"]

In [8]:
# Convert X and y to numpy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build NN
model = keras.Sequential([
    keras.layers.Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")  # binary classification
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


C:\Users\91880\AppData\Roaming\Python\Python313\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# Train
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)
logger.log('model training completed')

# Evaluate
loss, acc = model.evaluate(X_test, y_test)
logger.log(f'models test accuracy: {acc}')
print("Neural Network Accuracy:", acc)

Epoch 1/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7015 - loss: 5.1546 - val_accuracy: 0.7764 - val_loss: 0.9066
Epoch 2/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7309 - loss: 2.2887 - val_accuracy: 0.8078 - val_loss: 1.3687
Epoch 3/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7329 - loss: 2.0556 - val_accuracy: 0.8018 - val_loss: 1.5622
Epoch 4/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7428 - loss: 2.3838 - val_accuracy: 0.5696 - val_loss: 1.7189
Epoch 5/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7417 - loss: 1.7426 - val_accuracy: 0.7493 - val_loss: 0.6680
Epoch 6/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7460 - loss: 1.4025 - val_accuracy: 0.6215 - val_loss: 1.2229
Epoch 7/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7483 - loss: 1.5984 - val_accuracy: 0.7991 - val_loss: 2.4314
Epoch 8/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7552 - loss: 1.0784 - val_accuracy: 0.

In [11]:
today = datetime.today().strftime("%Y\\%m\\%d")
timestamp = datetime.today().strftime("%H%M")

# make directory path only
dir_path = fr"saved-models\{today}"
os.makedirs(dir_path, exist_ok=True)

# file path inside that directory
file_path = fr"{dir_path}\churn_nn_{timestamp}.keras"

try:
    model.save(file_path)
    print(f"Model saved at: {file_path}")
    logger.log(f'model succesfully saved at {file_path}')
except:
    logger.log('failed to save the model')


Model saved at: saved-models\2025\08\24\churn_nn_1907.keras
